# Local Torrent Downloader with Auto-Archive

This script downloads torrents to local Colab storage (~70GB) and automatically creates 700MB archives for easy download.

⚠️ **Important**: For research and educational purposes only. Please respect copyright laws and use only for legal content.

## 1. Install Dependencies

In [ ]:
# Install dependencies
!apt-get update
!apt-get install -y libtorrent-rasterbar-dev p7zip-full
!pip install libtorrent

# Check installation
import libtorrent as lt
print(f"✅ LibTorrent version: {lt.version}")
print("✅ 7zip installed for archiving")
print("✅ Installation completed successfully!")
print(f"📁 Available local storage: ~70GB in /content/")

## 2. Setup Local Storage

In [ ]:
import os
import shutil

# Create local directories
torrent_dir = '/content/torrents'
archive_dir = '/content/archives'

os.makedirs(torrent_dir, exist_ok=True)
os.makedirs(archive_dir, exist_ok=True)

# Check available space
def get_disk_usage(path):
    total, used, free = shutil.disk_usage(path)
    return {
        'total': total // (1024**3),  # GB
        'used': used // (1024**3),
        'free': free // (1024**3)
    }

usage = get_disk_usage('/content')
print(f"💾 Disk usage: {usage['used']}GB used / {usage['total']}GB total")
print(f"💾 Available space: {usage['free']}GB")
print(f"📁 Torrent directory: {torrent_dir}")
print(f"📦 Archive directory: {archive_dir}")

## 3. Enhanced Torrent Downloader with Auto-Archive

In [ ]:
import libtorrent as lt
import time
import os
import subprocess
import shutil
from IPython.display import clear_output
from google.colab import files

class LocalTorrentDownloader:
    def __init__(self, download_path, archive_path):
        self.download_path = download_path
        self.archive_path = archive_path
        self.session = lt.session()
        self.session.listen_on(6881, 6891)
        
    def download_torrent(self, torrent_file_or_magnet, auto_archive=True):
        """Download torrent and optionally create archives"""
        try:
            # Check if it's a magnet link or file
            if torrent_file_or_magnet.startswith('magnet:'):
                handle = lt.add_magnet_uri(self.session, torrent_file_or_magnet, {
                    'save_path': self.download_path
                })
            else:
                # Load torrent file
                info = lt.torrent_info(torrent_file_or_magnet)
                handle = self.session.add_torrent({
                    'ti': info,
                    'save_path': self.download_path
                })
            
            torrent_name = handle.name()
            print(f"🚀 Starting download: {torrent_name}")
            
            # Progress monitoring
            while not handle.is_seed():
                status = handle.status()
                
                clear_output(wait=True)
                print(f"📁 Torrent: {torrent_name}")
                print(f"📊 Progress: {status.progress * 100:.1f}%")
                print(f"⬇️ Download: {status.download_rate / 1000:.1f} KB/s")
                print(f"⬆️ Upload: {status.upload_rate / 1000:.1f} KB/s")
                print(f"👥 Peers: {status.num_peers} | Seeds: {status.num_seeds}")
                
                # Show disk usage
                usage = get_disk_usage('/content')
                print(f"💾 Disk: {usage['used']}GB used / {usage['free']}GB free")
                
                if status.state == lt.torrent_status.downloading:
                    print("🔄 Status: Downloading")
                elif status.state == lt.torrent_status.finished:
                    print("✅ Status: Download Finished")
                    break
                elif status.state == lt.torrent_status.seeding:
                    print("🌱 Status: Seeding")
                    break
                
                time.sleep(2)
            
            print(f"\n✅ Download completed: {torrent_name}")
            
            # Auto-archive if requested
            if auto_archive:
                print("📦 Creating archives...")
                self.create_archives(torrent_name)
                
        except Exception as e:
            print(f"❌ Download error: {e}")
    
    def create_archives(self, torrent_name):
        """Create 700MB archives from downloaded content"""
        try:
            source_path = os.path.join(self.download_path, torrent_name)
            if not os.path.exists(source_path):
                print(f"❌ Source path not found: {source_path}")
                return
            
            # Clean torrent name for filename
            clean_name = "".join(c for c in torrent_name if c.isalnum() or c in (' ', '-', '_')).strip()
            archive_name = os.path.join(self.archive_path, f"{clean_name}.7z")
            
            print(f"📦 Creating archive: {clean_name}.7z")
            print(f"📁 Source: {source_path}")
            
            # Create 700MB volumes using 7zip
            cmd = [
                '7z', 'a',
                '-v700m',  # 700MB volumes
                '-mx=5',   # Medium compression
                archive_name,
                source_path
            ]
            
            print("🔄 Compressing... This may take a while.")
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode == 0:
                print("✅ Archive created successfully!")
                self.list_archives()
                
                # Ask if user wants to delete original files
                print("\n🗑️ Delete original files to save space? (y/n)")
                # Note: In actual use, you'd want user input here
                # For now, we'll keep the files
                
            else:
                print(f"❌ Archive creation failed: {result.stderr}")
                
        except Exception as e:
            print(f"❌ Archive error: {e}")
    
    def list_files(self):
        """Show downloaded files"""
        print("\n📁 Downloaded files:")
        if not os.path.exists(self.download_path):
            print("No downloads yet.")
            return
            
        total_size = 0
        for root, dirs, files in os.walk(self.download_path):
            level = root.replace(self.download_path, '').count(os.sep)
            indent = ' ' * 2 * level
            folder_name = os.path.basename(root) if level > 0 else "Downloads"
            print(f"{indent}📁 {folder_name}/")
            
            sub_indent = ' ' * 2 * (level + 1)
            for file in files:
                file_path = os.path.join(root, file)
                size = os.path.getsize(file_path)
                size_mb = size / (1024 * 1024)
                total_size += size
                print(f"{sub_indent}📄 {file} ({size_mb:.1f} MB)")
        
        print(f"\n💾 Total size: {total_size / (1024**3):.2f} GB")
    
    def list_archives(self):
        """Show created archives"""
        print("\n📦 Created archives:")
        if not os.path.exists(self.archive_path):
            print("No archives yet.")
            return
            
        archives = [f for f in os.listdir(self.archive_path) if f.endswith('.7z')]
        if not archives:
            print("No archives found.")
            return
            
        # Group by base name (volumes)
        archive_groups = {}
        for archive in archives:
            base_name = archive.split('.7z')[0]
            if base_name not in archive_groups:
                archive_groups[base_name] = []
            archive_groups[base_name].append(archive)
        
        total_size = 0
        for base_name, volumes in archive_groups.items():
            print(f"\n📦 {base_name}:")
            volumes.sort()
            for volume in volumes:
                volume_path = os.path.join(self.archive_path, volume)
                size = os.path.getsize(volume_path)
                size_mb = size / (1024 * 1024)
                total_size += size
                print(f"  📄 {volume} ({size_mb:.1f} MB)")
        
        print(f"\n💾 Total archive size: {total_size / (1024**3):.2f} GB")
    
    def download_archives(self, archive_name=None):
        """Download archive files to local computer"""
        if archive_name:
            # Download specific archive volumes
            volumes = [f for f in os.listdir(self.archive_path) 
                      if f.startswith(archive_name) and f.endswith('.7z')]
            volumes.sort()
            
            print(f"📥 Downloading {len(volumes)} volumes for {archive_name}...")
            for volume in volumes:
                volume_path = os.path.join(self.archive_path, volume)
                print(f"⬇️ Downloading {volume}...")
                files.download(volume_path)
        else:
            # Download all archives
            archives = [f for f in os.listdir(self.archive_path) if f.endswith('.7z')]
            print(f"📥 Downloading {len(archives)} archive files...")
            for archive in archives:
                archive_path = os.path.join(self.archive_path, archive)
                print(f"⬇️ Downloading {archive}...")
                files.download(archive_path)
    
    def cleanup(self, keep_archives=True):
        """Clean up downloaded files"""
        if os.path.exists(self.download_path):
            shutil.rmtree(self.download_path)
            os.makedirs(self.download_path, exist_ok=True)
            print("🗑️ Download folder cleaned")
        
        if not keep_archives and os.path.exists(self.archive_path):
            shutil.rmtree(self.archive_path)
            os.makedirs(self.archive_path, exist_ok=True)
            print("🗑️ Archive folder cleaned")

# Create downloader instance
downloader = LocalTorrentDownloader(torrent_dir, archive_dir)
print("✅ Local torrent downloader ready!")
print("📁 Downloads: /content/torrents/")
print("📦 Archives: /content/archives/")

## 4. Download Torrent

In [ ]:
# Option 1: Download by magnet link
magnet_link = "magnet:?xt=urn:btih:HASH&dn=filename"  # Replace with your magnet link
# downloader.download_torrent(magnet_link)

# Option 2: Download by torrent file
print("📁 Select torrent file for download:")
uploaded = files.upload()

if uploaded:
    torrent_file = list(uploaded.keys())[0]
    print(f"📄 Uploaded file: {torrent_file}")
    
    # Start download with auto-archive
    downloader.download_torrent(torrent_file, auto_archive=True)
else:
    print("❌ No file uploaded")

## 5. Manual Archive Creation

In [ ]:
# Create archives manually for existing downloads
downloader.list_files()

# If you want to archive a specific torrent:
# torrent_name = "YourTorrentName"  # Replace with actual torrent name
# downloader.create_archives(torrent_name)

## 6. View and Download Archives

In [ ]:
# Show all archives
downloader.list_archives()

# Show current disk usage
usage = get_disk_usage('/content')
print(f"\n💾 Current disk usage: {usage['used']}GB / {usage['total']}GB")
print(f"💾 Available space: {usage['free']}GB")

## 7. Download Archives to Computer

In [ ]:
# Download all archives
downloader.download_archives()

# Or download specific archive by name:
# downloader.download_archives("YourArchiveName")

## 8. Cleanup

In [ ]:
# Clean up downloaded files (keep archives)
# downloader.cleanup(keep_archives=True)

# Clean up everything
# downloader.cleanup(keep_archives=False)

print("🧹 Uncomment the lines above to clean up files")

## 📝 Notes

### Storage
- **70GB** local storage available in `/content/`
- Files are automatically archived into **700MB volumes**
- Archives can be downloaded individually
- Storage is cleared when Colab session restarts

### Archive Format
- Uses 7zip compression with 700MB volumes
- Files named: `archive.7z.001`, `archive.7z.002`, etc.
- Extract with 7zip, WinRAR, or similar tools

### Advantages over Google Drive version
- **More storage**: 70GB vs 15GB
- **Faster downloads**: Local storage is faster
- **Easy archiving**: Auto-split into manageable chunks
- **No cloud authorization**: Works without Drive mounting

### Limitations
- Files lost when session restarts (download archives first!)
- Individual file download limit: ~2GB via Colab
- Session timeout: 12 hours maximum

### Legal Notice
**For research and educational purposes only.** Use only for legal content and respect copyright laws.